In [2]:
import numpy as np
import pandas as pd
import ast
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load the datasets
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

# Merge movies and credits on the 'title' column
movies = movies.merge(credits, on='title')

# Keep only the important columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Drop rows with missing values
movies.dropna(inplace=True)

# Remove duplicates if any
movies.drop_duplicates(inplace=True)

# Convert stringified lists into actual lists using 'ast.literal_eval'
def convert(text):
    return [i['name'] for i in ast.literal_eval(text)]

# Apply the conversion function to the 'genres' and 'keywords' columns
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

# Handle 'cast' column by keeping only the top 3 cast members
def convert_cast(text):
    return [i['name'] for i in ast.literal_eval(text)[:3]]

movies['cast'] = movies['cast'].apply(convert_cast)

# Extract the director's name from the 'crew' column
def fetch_director(text):
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            return [i['name']]
    return []

movies['crew'] = movies['crew'].apply(fetch_director)

# Convert 'overview' column into a list of words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# Remove spaces from names in 'cast', 'crew', 'genres', and 'keywords'
def remove_space(L):
    return [i.replace(" ", "") for i in L]

movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

# Concatenate all relevant columns into a single 'tags' column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# Create a new dataframe with 'movie_id', 'title', and 'tags' columns
new_df = movies[['movie_id', 'title', 'tags']]

# Convert the list of tags into a single string for each movie
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

# Convert all tags to lowercase
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

# Initialize the Porter Stemmer for stemming
ps = PorterStemmer()

# Define a function to apply stemming to the 'tags' column
def stems(text):
    return " ".join([ps.stem(word) for word in text.split()])

# Apply stemming to the 'tags' column
new_df['tags'] = new_df['tags'].apply(stems)

# Vectorize the 'tags' column using CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new_df['tags']).toarray()

# Compute the cosine similarity between the vectors
similarity = cosine_similarity(vector)

# Function to recommend movies based on similarity
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

C:\Users\HP\AppData\Local\Temp\ipykernel_3852\2133949468.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\HP\AppData\Local\Temp\ipykernel_3852\2133949468.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
C:\Users\HP\AppData\Local\Temp\ipykernel_3852\2133949468.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [3]:
# Example usage
recommend('Spider-Man 2')

# Save the data and model using pickle
pickle.dump(new_df, open('artifacts/movie_list.pkl', 'wb'))
pickle.dump(similarity, open('artifacts/similarity.pkl', 'wb'))

Spider-Man 3
Spider-Man
The Amazing Spider-Man
Iron Man 2
Superman


██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [1]:
import requests
import json

# List of API keys and their usage count
api_keys = ['7058f3e1', '38bad860', 'fd237622', 'e189e68f', 'b4d0b532', '4bede092', 'a9e2d1c']  # Add more keys as needed
api_key_usage = {key: 0 for key in api_keys}  # Dictionary to track API key usage
request_limit = 1000  # Set to OMDb's free daily limit

# Function to fetch movie details from the OMDB API using multiple keys
def movie_detail(movie_name):
    global api_keys, api_key_usage
    
    # Loop through each API key until one works or limit is hit
    for api_key in api_keys:
        # Check if the current API key has reached its limit
        if api_key_usage[api_key] < request_limit:
            url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
            
            try:
                response = requests.get(url)
                
                # If request is successful, check the status code
                if response.status_code == 200:
                    movie_data = response.json()
                    
                    # Increment the usage count for the current API key
                    api_key_usage[api_key] += 1
                    
                    # Check if the 'Title' key exists in the response data
                    if 'Title' in movie_data:
                        return movie_data
                    else:
                        # Log if there is a problem with the API response
                        print(f"API key {api_key} returned no 'Title' in the response. Response: {movie_data}")
                else:
                    print(f"API key {api_key} failed with status code: {response.status_code}")
                    
            except requests.exceptions.RequestException as e:
                print(f"Error fetching data with API key {api_key}: {e}")
    
    # If no keys worked or all hit the limit, return default values
    return {
        'Title': 'N/A',
        'Year': 'N/A',
        'Runtime': 'N/A',
        'imdbRating': 'N/A',
        'Poster': 'N/A'
    }

# Example usage
movie_data = movie_detail("Inception")
print(movie_data)


{'Title': 'Inception', 'Year': '2010', 'Rated': 'PG-13', 'Released': '16 Jul 2010', 'Runtime': '148 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Christopher Nolan', 'Writer': 'Christopher Nolan', 'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page', 'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.', 'Language': 'English, Japanese, French', 'Country': 'United States, United Kingdom', 'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '87%'}, {'Source': 'Metacritic', 'Value': '74/100'}], 'Metascore': '74', 'imdbRating': '8.8', 'imdbVotes': '2,592,712', 'imdbID'

In [5]:
import requests

# Function to fetch movie details from the OMDB API
api_keys = ['7058f3e1', '38bad860', 'fd237622', 'e189e68f', 'b4d0b532', '4bede092', 'a9e2d1c']  # Add more keys as needed

# Function to fetch movie details from the OMDB API using multiple keys
def movie_detail(movie_name):
    global api_keys
    # Loop through each API key until one works
    for api_key in api_keys:
        url = f"http://www.omdbapi.com/?t={movie_name}&apikey={api_key}"
        try:
            response = requests.get(url)
            # If request is successful, check the status code
            if response.status_code == 200:
                movie_data = response.json()
                # Check if the 'Title' key exists in the response data
                if 'Title' in movie_data:
                    print(f"API key {api_key}")
                    return movie_data
                else:
                    # Log if there is a problem with the API response
                    print(f"API key {api_key} returned no 'Title' in the response. Response: {movie_data}")
            else:
                print(f"API key {api_key} failed with status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data with API key {api_key}: {e}")
    # If no keys worked, return default values
    return {
        'Title': 'N/A',
        'Year': 'N/A',
        'Runtime': 'N/A',
        'imdbRating': 'N/A',
        'Poster': 'N/A'
    }

In [6]:
# Example usage
movie_data = movie_detail("Inception")
print(movie_data)

API key 7058f3e1
{'Title': 'Inception', 'Year': '2010', 'Rated': 'PG-13', 'Released': '16 Jul 2010', 'Runtime': '148 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Christopher Nolan', 'Writer': 'Christopher Nolan', 'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page', 'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.', 'Language': 'English, Japanese, French', 'Country': 'United States, United Kingdom', 'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '87%'}, {'Source': 'Metacritic', 'Value': '74/100'}], 'Metascore': '74', 'imdbRating': '8.8', 'imdbVotes': '2,5